# Using VGG16 for Face Recognition

Loading the VGG16 Model

In [ ]:
from keras.applications import VGG16

img_rows = 224
img_cols = 224

model = VGG16(weights='imagenet', include_top = False, input_shape=(img_rows, img_cols, 3))

for  layer in model.layers:
    layer.trainable = False

Function returns fc_head

In [ ]:
def fully_connected_head(model, num_classes):
    fc_head = model.output
    fc_head = Flatten(name = "flatten")(fc_head)
    fc_head = Dense(1024, activation='relu')(fc_head)
    fc_head = Dense(512, activation='relu')(fc_head)
    fc_head = Dense(256, activation='relu')(fc_head)
    fc_head = Dense(num_classes, activation='softmax')(fc_head)
    return fc_head

Adding fc_head to VGG16

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.models import Model

# assign no. of classes available to num_classes
num_classes = 10

fc_head = fully_connected_head(model, num_classes)

model = Model(inputs = model.input, outputs = fc_head)

Loading dataset and performing data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# assign path of your training dataset to train_data_dir
# assign path of your validation dataset to validation_data_dir
train_data_dir = 'human_face/train/'
validation_data_dir = 'human_face/test/'
 
train_data_aug = ImageDataGenerator(rescale=1./255, rotation_range=45, 
                                             width_shift_range=0.3, height_shift_range=0.3,
                                             horizontal_flip=True, fill_mode='nearest')
 
validation_data_aug = ImageDataGenerator(rescale=1./255)

batch_size = 32
 
train_data_augmentation = train_data_aug.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols),
                                                    batch_size=batch_size, class_mode='categorical')
 
validation_data_augmentation = validation_data_aug.flow_from_directory(validation_data_dir, target_size=(img_rows, img_cols),
                                                              batch_size=batch_size, class_mode='categorical')

Train/Fit our model

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image

                     
checkpoint = ModelCheckpoint("face_rec_vgg16.h5", monitor="val_loss",
                              mode="min", save_best_only = True, verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3,
                          verbose = 1, restore_best_weights = True)

callbacks = [earlystop, checkpoint]
 
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 450
nb_validation_samples = 50

epochs = 5
batch_size = 16

history = model.fit_generator(train_data_augmentation, steps_per_epoch = nb_train_samples // batch_size,
                              epochs = epochs, callbacks = callbacks, validation_data = validation_data_augmentation,
                              validation_steps = nb_validation_samples // batch_size)

Load our classifier

In [ ]:
from keras.models import load_model

classifier = load_model('face_rec_vgg16.h5')

Test our model

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

human_face_dict = {"[0]": "Ariel_Sharon", 
                      "[1]": "Colin_Powell",
                      "[2]": "Donald_Rumsfeld",
                      "[3]": "George_W_Bush",
                      "[4]": "Gerhard_Schroeder",
                      "[5]": "Hugo_Chavez",
                      "[6]": "Jean_Chretien",
                      "[7]": "John_Ashcroft",
                      "[8]": "Junichiro_Koizumi",
                      "[9]": "Tony_Blair"}

human_face_dict_n = {"n0": "Ariel_Sharon", 
                      "n1": "Colin_Powell",
                      "n2": "Donald_Rumsfeld",
                      "n3": "George_W_Bush",
                      "n4": "Gerhard_Schroeder",
                      "n5": "Hugo_Chavez",
                      "n6": "Jean_Chretien",
                      "n7": "John_Ashcroft",
                      "n8": "Junichiro_Koizumi",
                      "n9": "Tony_Blair"}

def draw_test(name, pred, im):
    human = human_face_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, human, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + human_face_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("human_face/test/")
    if input_im is None:
        continue
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
    
cv2.destroyAllWindows()